In [2]:
from __future__ import print_function

from keras.engine import Model
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense

import numpy as np

from pre_processing import load_data

Using TensorFlow backend.


In [3]:
# Load the data.
(x_train, y_train), (x_test, y_test), vocab = load_data(train_fraction=0.7, speakers=['s1'])
print('Number of words for training: ', x_train.shape[0])
print('Number of words for testing: ', x_test.shape[0])
print('Frames per word: ', x_train.shape[1])
print('Features per frame: ', x_train.shape[2])

Skipped 5 empty data files.
Number of words for training:  4213
Number of words for testing:  1806
Frames per word:  6
Features per frame:  512


In [4]:
# Build the LSTM model.
model = Sequential()
model.add(LSTM(128, input_shape=x_train[0].shape, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(vocab), activation='tanh'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [5]:
# Train the model.
model.fit(x_train, y_train, batch_size=32, epochs=30, verbose=1)

Epoch 1/30
4213/4213 [==============================] - 3s - loss: 0.0170 - acc: 0.3321      

In [6]:
# Test the model.
score, acc = model.evaluate(x_test, y_test, batch_size=32)
print('Test score: ', score)
print('Test accuracy: ', acc)

1664/1806 [==========================>...] - ETA: 0sTest score:  0.00617493054397
Test accuracy:  0.805094130676


In [9]:
# Try matching some results.
frames_per_word = x_test.shape[1]
features_per_frame = x_test.shape[2]

RESULTS_TO_CHECK = 50
MATCHES_PER_WORD = 4

for i in xrange(RESULTS_TO_CHECK):
    pred = model.predict(x_test[i].reshape(1, frames_per_word, features_per_frame))
    predicted_indexes = np.argsort(pred.reshape(len(vocab)))[::-1][:MATCHES_PER_WORD]
    correct_index = np.argsort(y_test[i])[::-1][0]

    correct_word = vocab[correct_index]
    predicted_words = [vocab[i] for i in predicted_indexes]

    print('{} : {}'.format(correct_word, predicted_words))

l : ['now', 'l', 'f', 'again']
b : ['p', 'b', 'z', 'v']
soon : ['soon', 'seven', 'three', 'four']
red : ['green', 'with', 'b', 'u']
six : ['six', 'q', 'k', 'z']
green : ['green', 'soon', 'g', 'lay']
c : ['t', 'd', 'c', 'e']
blue : ['blue', 'g', 'with', 'e']
again : ['again', 'six', 'five', 'f']
y : ['by', 'y', 'one', 'red']
c : ['c', 'z', 'set', 'i']
four : ['four', 'soon', 'in', 'o']
by : ['by', 'in', 'place', 'white']
four : ['four', 'one', 'o', 'blue']
at : ['by', 'p', 'e', 'bin']
red : ['red', 'in', 'n', 'x']
red : ['red', 'y', 'nine', 'g']
two : ['soon', 'zero', 'q', 'c']
please : ['please', 'p', 'i', 'six']
n : ['f', 'l', 'n', 'c']
in : ['in', 'h', 'i', 'with']
seven : ['seven', 'h', 'f', 't']
blue : ['blue', 'o', 'y', 'sp']
now : ['at', 'in', 'now', 'j']
bin : ['bin', 'v', 'eight', 'lay']
again : ['again', 'five', 'q', 'eight']
now : ['now', 'again', 'q', 'six']
z : ['seven', 'z', 't', 'six']
again : ['again', 'one', 'd', 'five']
green : ['green', 'y', 'three', 'g']
by : ['eight